In [2]:
import requests
import pandas as pd

def make_api_call(base_url, endpoint="", method="GET", **kwargs):
    # Construct the full URL
    full_url = f'{base_url}{endpoint}'

    # Make the API call
    response = requests.request(method=method, url=full_url, **kwargs)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response
    else:
        # If the request was not successful, raise an exception with the error message
        raise Exception(f'API request failed with status code {response.status_code}: {response.text}')

# Get Binance historical data function
def get_binance_historical_data(symbol, interval, start_date, end_date):
    # Define basic parameters for call
    base_url = 'https://fapi.binance.com'
    endpoint = '/fapi/v1/klines'
    method = 'GET'
    
    # Set the start time parameter in the params dictionary
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1500,
        'startTime': start_date,
        'endTime': end_date
    }

    # Make initial API call to get candles
    response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

    candles_data = []

    while len(response.json()) > 0:
        # Append the received candles to the list
        candles_data.extend(response.json())

        # Update the start time for the next API call
        params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms
        
        if params['endTime'] != "" and params['startTime'] > params['endTime']:
            break

        # Make the next API call
        response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

    # Wrap the candles data as a pandas DataFrame
    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
               'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    dtype = {
        'open_time': 'datetime64[ms, Asia/Jerusalem]',
        'open': 'float64',
        'high': 'float64',
        'low': 'float64',
        'close': 'float64',
        'volume': 'float64',
        'close_time': 'datetime64[ms, Asia/Jerusalem]',
        'quote_asset_volume': 'float64',
        'number_of_trades': 'int64',
        'taker_buy_base_asset_volume': 'float64',
        'taker_buy_quote_asset_volume': 'float64',
        'ignore': 'float64'
    }
    
    df = pd.DataFrame(candles_data, columns=columns)
    df = df.astype(dtype)

    return df


## Download Bitcoin and Ethereum historical data

In [3]:
from datetime import datetime
symbol_bitcoin = 'BTCUSDT'
symbol_ethereum = 'ETHUSDT'
interval = '1h'
start = {'year': 2023, 'month': 6, 'day': 16}
end = {'year': 2024, 'month': 6, 'day': 17}
start_date = int(datetime(**start).timestamp() * 1000)
end_date = int(datetime(**end).timestamp() * 1000)


df_btc = get_binance_historical_data(symbol_bitcoin, interval, start_date, end_date)
df_eth = get_binance_historical_data(symbol_ethereum, interval, start_date, end_date)

## Description

In [4]:
df_btc.describe()

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
count,8809.000000,8809.000000,8809.000000,8809.000000,8809.000000,8.809000e+03,8.809000e+03,8809.000000,8.809000e+03,8809.0
mean,44415.099205,44571.368101,44253.585015,44419.749415,12770.593399,5.603973e+08,1.435446e+05,6352.939967,2.785471e+08,0.0
std,15948.579328,16030.119080,15859.237571,15949.054301,15395.367754,6.954762e+08,1.377072e+05,7739.377884,3.461610e+08,0.0
min,25008.200000,25125.000000,24581.000000,25008.300000,736.925000,1.980208e+07,1.213900e+04,387.830000,1.042956e+07,0.0
25%,29521.100000,29596.900000,29450.200000,29523.700000,4895.378000,1.949192e+08,6.443900e+04,2395.963000,9.593254e+07,0.0
50%,41804.800000,41922.500000,41679.000000,41805.200000,7989.908000,3.443783e+08,1.011890e+05,3965.564000,1.711666e+08,0.0
75%,62853.700000,63139.000000,62549.200000,62864.000000,14768.874000,6.528628e+08,1.721770e+05,7385.265000,3.230729e+08,0.0
max,73644.500000,73881.400000,73270.200000,73644.600000,355275.447000,1.434873e+10,2.304598e+06,183487.620000,7.216176e+09,0.0


In [5]:
df_eth.describe()

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
count,8809.000000,8809.000000,8809.000000,8809.000000,8.809000e+03,8.809000e+03,8.809000e+03,8809.000000,8.809000e+03,8809.0
mean,2434.813695,2444.444616,2424.489079,2435.032094,1.042680e+05,2.622480e+08,1.196079e+05,51574.867603,1.296249e+08,0.0
std,739.999738,745.005093,734.482716,740.057992,1.115840e+05,3.098008e+08,1.101399e+05,55123.033516,1.522852e+08,0.0
min,1528.170000,1530.760000,1492.660000,1528.180000,7.687973e+03,1.255732e+07,8.833000e+03,3047.718000,5.072534e+06,0.0
25%,1837.750000,1842.090000,1834.040000,1837.900000,4.386083e+04,9.642072e+07,5.362800e+04,21506.682000,4.747755e+07,0.0
50%,2233.830000,2241.940000,2223.580000,2233.830000,7.116939e+04,1.690298e+08,8.749700e+04,35280.910000,8.378940e+07,0.0
75%,3084.980000,3099.270000,3069.240000,3085.080000,1.214815e+05,3.084670e+08,1.458300e+05,60501.640000,1.540702e+08,0.0
max,4072.930000,4098.640000,4066.800000,4072.920000,1.759250e+06,5.265259e+09,1.488208e+06,911494.585000,2.637059e+09,0.0
